In [1]:
from __future__ import print_function
import tensorflow as tf
import numpy as np
import cv2
import random
import os

In [17]:
class create_input:
    def load_image(self, path):
        # load image and resize into (224, 224)
        image = tf.io.read_file(path)
        image = tf.io.decode_png(image, channels=3)
        image = tf.cast(image, tf.float32)
        image = tf.image.resize(image, (224, 224))
        return image

    def cut_out(self, img):
        # choose one or three rectangle with random size(up to 1/5 image size)
        height = tf.shape(img)[0]
        width = tf.shape(img)[1]
        h = tf.random.uniform([], int(height/10), int(height/5), tf.int32)
        w = tf.random.uniform([], int(width/10), int(width/5), tf.int32)

        x1 = tf.random.uniform([], 0, height+1 - h, tf.int32)
        y1 = tf.random.uniform([], 0, width+1 - w, tf.int32)

        mean = int(tf.reduce_mean(img))
        erase_area = tf.cast(tf.fill([h, w, 3], 230), tf.uint8)
        erase_img = np.array(img)
        erase_img[x1:x1+h, y1:y1+w, :] = erase_area

        return erase_img

    def WhiteNoise(self, img):
        noise = tf.random.normal(tf.shape(img), mean=0.0, stddev=20.0, dtype=tf.float32)
        noise_img = noise+img
        noise_img = tf.clip_by_value(noise_img, 0.0, 255.0)

        return noise_img

    def SaltNoise(self, img):
        # add salt and pepper noise(1/16 area)
        pts_x_white = np.random.randint(0, tf.shape(img)[0]-1, int(tf.shape(img)[0]**2/16))
        pts_y_white = np.random.randint(0, tf.shape(img)[1]-1, int(tf.shape(img)[0]**2/16))

        pts_x_black = np.random.randint(0, tf.shape(img)[0]-1, int(tf.shape(img)[0]**2/16))
        pts_y_black = np.random.randint(0, tf.shape(img)[1]-1, int(tf.shape(img)[0]**2/16))

        img = np.array(img)
        img[pts_x_white, pts_y_white] = [255, 255, 255]
        img[pts_x_black, pts_y_black] = [0, 0, 0]

        return img

    def fit(self, path):
        img = self.load_image(path)

        augs = [self.cut_out, self.WhiteNoise, self.SaltNoise]

        num = int(tf.random.uniform([], minval=1, maxval=len(augs), dtype=tf.int64))
        selected_augs = random.sample(augs, num)
        for aug in selected_augs:
            img = aug(img)

        return img

In [21]:
# create augmented image
img_path = "/home/naoki/Documents/GAN_practice/data/test/"
save_path = "/home/naoki/Documents/GAN_practice/data/aug/test/"
files = os.listdir(img_path)
files_file = [f for f in files if os.path.isfile(os.path.join(img_path, f))]  #  all image names
random.shuffle(files_file)
files_path = [img_path+files_file[i] for i in range(len(files_file))]  #  all image path
save_files_path = [save_path+files_file[i] for i in range(len(files_file))]  #  all image path

for (path, sv_path) in zip(files_path, save_files_path):
    fit_img = np.array(create_input().fit(path))
    fit_img = cv2.cvtColor(fit_img, cv2.COLOR_BGR2RGB)

    cv2.imwrite(sv_path, fit_img)

In [36]:
aug = data_aug()
path = "/home/naoki/Documents/NEW-GAME-classification/data/2.png"
fit_img = data_aug().fit(path)
fit_img = cv2.cvtColor(fit_img, cv2.COLOR_BGR2RGB)
cv2.imwrite("/home/naoki/Pictures/fit_noise.png", fit_img)

True

In [29]:
aug = data_aug()
image = data_aug().load_image("/home/naoki/Documents/NEW-GAME-classification/data/2.png")
image = aug.WhiteNoise(image)
image = np.array(image)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
cv2.imwrite("/home/naoki/Pictures/white_noise.png", image)

True

In [39]:
aug = data_aug()
image = aug.cut_out("/home/naoki/Documents/NEW-GAME-classification/data/2.png")
image = np.array(image)
cv2.imwrite("/home/naoki/Pictures/cut_out.png", image)

True

In [18]:
class Hellos:
    def hello(self, path):
        print("Hello")
    def hello_n(self,n):
        for i in range(n):
            self.hello(1)

In [19]:
h = Hellos()
h.hello_n(3)

Hello
Hello
Hello


In [5]:
class data_aug:
    def resize(self, input_image, real_image, height, width):
        input_image = tf.image.resize(input_image, [height, width], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
        real_image = tf.image.resize(real_image, [height, width], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

        return input_image, real_image

    def random_crop(self, input_image, real_image):
        stack_image = tf.stack([input_image, real_image], axis=0)
        cropped_image = tf.image.random_crop(stack_image, size=[2, 256, 256, 3])

        return cropped_image[0], cropped_image[1]

    def random_jitter(input_image, real_image):
        input_image, real_image = self.resize(input_image, real_image, 286, 286)
        input_image, real_image = self.random_crop(input_image, real_image)

        if tf.random.uniform(()) > 0.5:
            input_image = tf.image.flip_left_right(input_image)
            real_image = tf.image.flip_left_right(real_image)

        return input_image, real_image